In [1]:
import sys
import operator
from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf
import re
import unicodedata
from operator import add
from itertools import combinations

In [2]:
spark = SparkSession \
            .builder \
            .appName("clustering") \
            .getOrCreate()
sc = spark.sparkContext

In [3]:
data = spark.read.csv('spofity_cleaned.csv', header = True)

In [4]:
names = data.rdd.map(lambda x: x['track_name']).distinct()\
                .map(lambda x: (x, x.strip()))\
                .map(lambda x: (x[0], x[1].lower()))\
                .map(lambda x: (x[0], re.sub(r'[^\w\s]','', x[1])))\
                .map(lambda x: (x[0], unicodedata.normalize('NFC',x[1])))\
                .map(lambda x: (x[0], x[1].split(' ')))\
                .map(lambda x: (x[0], list(filter(None, x[1])))).cache()

In [7]:
print(names.take(10))

[("Don't Let Me Be Lonely Tonight", ['dont', 'let', 'me', 'be', 'lonely', 'tonight']), ('Dis-moi Monsieur Gordon Cooper', ['dismoi', 'monsieur', 'gordon', 'cooper']), ('Les bisous des bisounours', ['les', 'bisous', 'des', 'bisounours']), ('The Hanging (Maverick - Original Motion Picture Score) - Remastered', ['the', 'hanging', 'maverick', 'original', 'motion', 'picture', 'score', 'remastered']), ('Diane', ['diane']), ('Ladies And Gentlemen', ['ladies', 'and', 'gentlemen']), ('Pourquoi aller plus loin ?', ['pourquoi', 'aller', 'plus', 'loin']), ('Silence on tourne', ['silence', 'on', 'tourne']), ('Sailing', ['sailing']), ('Dancing with Gene', ['dancing', 'with', 'gene'])]


In [5]:
all_names=names.collect()
name_dict ={}
for each in all_names:
    name_dict[each[0]]=each[1]

In [6]:
tokens = names.map(lambda x: x[1]).collect()

In [7]:
shingles = []
for each in tokens:
    shingles.extend(each)
shingles = sorted(list(set(shingles)))
rep_shingles = {}
i = 1
for each in shingles:
    rep_shingles[each] = i
    i = i+1

In [8]:
print(len(shingles))

9166


In [23]:
print(rep_shingles)

{'001': 1, '0026': 2, '0027': 3, '003': 4, '004554': 5, '007': 6, '0078': 7, '013': 8, '016': 9, '026176': 10, '033': 11, '0375': 12, '0409': 13, '041': 14, '0410': 15, '041500': 16, '051': 17, '05423': 18, '05425': 19, '05426': 20, '05430': 21, '05434': 22, '05435': 23, '05436': 24, '05456': 25, '05457': 26, '05458': 27, '05879': 28, '059': 29, '06023': 30, '06211': 31, '063': 32, '06516': 33, '06854': 34, '06931': 35, '06976': 36, '07016': 37, '07036': 38, '0717': 39, '07482': 40, '083': 41, '0876': 42, '09136': 43, '09144': 44, '09285': 45, '09563': 46, '09985': 47, '1': 48, '10': 49, '100': 50, '1000': 51, '1003': 52, '10070': 53, '101': 54, '10107': 55, '10129': 56, '102': 57, '1021': 58, '10291': 59, '10292': 60, '10295': 61, '10296': 62, '103': 63, '10309': 64, '1039': 65, '103rd': 66, '104': 67, '10433': 68, '10469': 69, '105': 70, '10509': 71, '10586': 72, '105meat': 73, '106': 74, '1061': 75, '10614': 76, '1064': 77, '10663': 78, '107': 79, '10724': 80, '10734': 81, '10791': 

In [8]:
trans_names = names.mapValues(lambda x: [rep_shingles[i] for i in x]).cache()

In [12]:
print(trans_names.take(20))

[("Don't Let Me Be Lonely Tonight", [13101, 25624, 28060, 4644, 26321, 45122]), ('Dis-moi Monsieur Gordon Cooper', [12746, 29318, 18632, 10151]), ('Les bisous des bisounours', [25603, 5363, 12086, 5362]), ('The Hanging (Maverick - Original Motion Picture Score) - Remastered', [44451, 19671, 27927, 32132, 29602, 33877, 39142, 36834]), ('Diane', [12395]), ('Ladies And Gentlemen', [24917, 2474, 17964]), ('Pourquoi aller plus loin ?', [34752, 2051, 34314, 26270]), ('Silence on tourne', [40512, 31843, 45289]), ('Sailing', [38409]), ('Dancing with Gene', [11287, 48956, 17916]), ('Penn Station - Seesaw', [33416, 42338, 39360]), ('Super Insomniaque', [43161, 22107]), ('Le Petit Poucet 2', [25369, 33674, 34737, 367]), ('La petite diligence', [24845, 33675, 12535]), ('Grosse Fille Ne Pleure Pas - Remastered', [19016, 16139, 30463, 34284, 33010, 36834]), ("Dis-moi qu'tu m'aimes", [12746, 35895, 27106]), ('Forgotten Dreams', [16780, 13404]), ('Daivapithave', [11191]), ('Shri Gayatri Mahamantra', [

In [9]:
def min_hash(values):
    sign = []
    for i in range(1, 101):
        hash_list = []
        for v in values:
            h_value = (2*v + 11*i) % 100
            hash_list.append(h_value)
        if hash_list!=[]:
            sign.append(min(hash_list))
    return sign
def band(item):
    band = []
    for i in range(0, 2):
        band.append((i, (item[0], item[1][i])))
    return band



In [10]:
def cluster(values):
    Output = {} 
    for x, y in values: 
        if tuple(y) in Output: 
            Output[tuple(y)].append((x)) 
        else: 
            Output[tuple(y)] = [(x)]
    return Output

In [11]:
def clean(l):
    output = []
    for each in l:
        if len(each[1])>=2:
            output.extend(each(1))
    return output

In [12]:
signiture = trans_names.mapValues(lambda x: min_hash(x))\
                        .mapValues(lambda x: [x[i:i+50] for i in range(0, 100, 50)])\
                        .flatMap(lambda x: band(x))\
                        .groupByKey()\
                        .map(lambda x: (x[0],list(x[1])))\
                        .mapValues(lambda x: cluster(x))\
                        .map(lambda x: list(x[1].items())).cache()

In [13]:
candidates = signiture.collect()

In [14]:

cand = []
for each in candidates:
    cand.extend(each)
candidate = []
for each in cand:
    if len(each[1])>=2:
        candidate.append(each[1])


In [15]:
print(len(candidate))

25744


In [16]:
def similarity(values):
    result = []
    permutations = list(combinations(values, 2))
    for each in permutations:
        u1 = each[0]
        u2 = each[1]
        m1 = name_dict[u1]
        m2 = name_dict[u2]
        inter = set(m1).intersection(set(m2))
        union = set(m1).union(set(m1))
        if len(union)!=0:
            simi = float(len(inter)/len(union))
            if simi>=0.9:
                result.append(each)
    return result

In [17]:
sc_cand = sc.parallelize(candidate).map(lambda x: similarity(x))

In [21]:
results = sc_cand.collect()
result = []
for each in results:
    if len(each)>0:
        result.append(each)
output = []
for i in result:
    clusters = []
    for each in i:
        clusters.append(each[0])
        clusters.append(each[1])
    clusters = list(set(clusters))
    output.append(clusters)
output2 = list(set([tuple(t) for t in output]))

In [22]:
for each in output:
    print(each)

['Menace', 'oasis', 'La La La La', 't-shirt', 'Royal', 'E.A.T.', 'LA LA', 'close', 'honestly', 'La Piedra', 'windows', '666', 'Homecoming Yodel', 'Again & Again', 'Film Burn', '666!', 'T-Shirt', 'progress', 'Progress', 'La', 'Friends Unknown', 'Suerte', 'Eat', 'Unknown', 'HOMECOMING', 'Close', 'La Solicitud', 'WHISTLE', 'MENACE', 'Source', 'Friends', 'TEAM', 'La La La', 'Windows', 'Oasis', 'Again', 'FRIENDS', 'La Pesca', 'La foule', 'L.A.', 'La Perrera', 'Honestly', 'Burn', 'La Barcheta', 'La La', 'Whistle', 'Piedra', 'La Suerte', 'La bicyclette', 'ROYAL', 'La source', 'La suerte', 'Homecoming', 'Team']
['Ladies And Gentlemen', 'Ladies and Gentlemen']
['Silence on tourne', 'Silence On Tourne', 'Silence on Tourne']
['17', 'Travelling', 'UNDERCOVER', 'Travelling Man', 'platinum', 'Leavin', 'Colour', 'Struggling', 'Dumb', 'Aria', 'Rebel Rebel', 'Listen Up', 'I.O.U.', 'Island.', 'Shot', 'Sailing', 'Michigan', 'Undercover', 'Armageddon Suite', 'Parsifal: Prelude', 'Rebel Bass', 'Supernova',

['New York', 'new york', 'Mon ange', 'New York New York', 'Mon Ange']
['Hit the Road Jack', 'Hit The Road Jack']
["Where's the Girl?", "Where's The Girl?"]
['When the Levee Breaks', 'When The Levee Breaks']
["Blowin' In The Wind", "Blowin' In the Wind", "Blowin' in the Wind"]
['Count on Me', 'Count On Me', 'COUNT ON ME']
['Rains it Pours', 'Rains It Pours', 'When It Rains It Pours']
['Crazy In Love', 'Crazy in Love']
['Abide With Me', 'Abide with Me']
['Chase the Devil', 'Chase The Devil']
['contigo peru', 'CONTIGO PERU']
['Peace In Liberia', 'Peace in Liberia']
['Girl Of My Dreams', 'Girl of My Dreams']
['The Shadow of Your Smile', 'The Shadow Of Your Smile']
['Late Night Tv', 'Late Night TV']
['Love Wont Leave Me Alone', "Love Won't Leave Me Alone"]
['Immigrants (We Get the Job Done)', 'Immigrants (We Get The Job Done)']
['Time of Your Song', 'Time Of Your Song']
["It's Lonely at the Top", 'Lonely at the Top', 'Lonely At the Top', 'Lonely At The Top']
['April in Paris', 'April In Par

['Middle of the Night', 'Middle Of The Night']
['U Got Me', 'U GOT ME', 'Me & U']
['Too Much Love', 'Love Too Much']
['Grocery Store', 'Reunion of Friends', 'Friends of Friends', 'Grocery Store Lady']
["Don't Put Me On", "Don't Put It On Me"]
['Strike Back', 'STRIKE BACK']
['Old Mcdonald', 'Old McDonald']
['Wait for It', 'Wait For It']
['Miss you', 'Still Miss You', 'Miss You', "Doin' Time", 'Blow Wind Blow', "Still Doin' Time", 'Wind Blow']
['Back Again', 'back again', 'Back Once Again']
['Two More Bottles of Wine', 'Two More Bottles Of Wine']
['Step by Step', 'Step By Step']
['Let It Snow! Let It Snow! Let It Snow!', 'Let It Snow']
['Perfect Picture', 'Picture Perfect']
['Ay! Ay! Amor!', 'The good', 'The Remix', "The Hunter's Path", 'The Good Good', 'Ay ay Amor', 'The Word', 'Say The Word', 'The Good', 'The Path', 'The Life', 'The Good Life', 'Thrive (The nightSHIFT Remix)']
['Run Away With You', 'Run Away with You']
['I Need A Woman', 'I Need a Woman']
['Call A Marine', 'Call a Mari

['Make Your Own Kind of Music', 'Make Your Own Kind Of Music']
['Mysterious Girl - Radio Edit', 'Girl - Radio Edit']
['What The World Needs Now Is Love', 'What the World Needs Now Is Love']
["I'll Give All My Love To You", "I'll Give All My Love to You"]
['Get in My Car', 'Get In My Car']
['EX - Acoustic', 'Ex - Acoustic']
['Sonata No. 16 in C Major', 'Sonata No. 16 in C']
['Monsieur le bon dieu', 'Monsieur Le Bon Dieu', 'Monsieur le bon Dieu', 'Monsieur le Bon Dieu']
['Come Go with Me', 'Come Go With Me']
['December Song [Bonus Track] - A Capella', 'December Song (A Capella Bonus Track)']
['Like Someone in Love', 'Like Someone In Love']
['What Child is This?', 'What Child Is This?', 'What Child Is This']
['Love Is Here to Stay', 'Love Is Here To Stay']
['Straighten up and Fly Right', 'Straighten Up And Fly Right']
['The Lady Is a Tramp', 'The Lady is a Tramp']
['Anywhere But Here', 'Whatcha Gonna Do', 'Whatcha Gonna Do?', 'Anywhere but Here']
['Will the Circle Be Unbroken?', 'Will The

In [27]:
print(len(output2))

4287


In [28]:
import csv
with open('sp_token_lsh.csv', "w") as f:
    w = csv.writer(f)
    w.writerows(output2)